# Inferring surface change by differencing NDIs

## Log in to Google Earth Engine

Once again, we will start by importing the libraries we will use and log in to Google Earth Engine.

In [1]:
import ee
import geemap
import geemap.chart as chart
import time

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.


## Search for scenes

This week...

 - https://www.aptnnews.ca/national-news/quebec-wildfires-cree-community-orders-evacuation-of-4k-due-to-heavy-smoke/
 - cwfis.cfs.nrcan.gc.ca/interactive-map

In [71]:

# Location - editable
# latitude = 51.202441           # Degrees of latitude
# longitude = -73.910748         # Degrees of longitude
# size = 12000                   # Size of AOI, in metres
# location_name = 'mistassini'   # recognisable name, to create a useful file name

latitude = -6.816            # Degrees of latitude
longitude = -59.119          # Degrees of longitude
size = 20000                 # Size of AOI, in metres
location_name = 'sucunduri'  # recognisable name, to create a useful file name

# Date range 1 - editable
date_1_start = '2022-06-01'
date_1_end = '2022-09-01'

# Date range 2 - editable
date_2_start = '2023-06-01'
date_2_end = '2023-09-01'


# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape


In [72]:
Map = geemap.Map()  # Create an empty Map
Map.addLayer(region, {}, "Search Region")  # Add our AOI
Map.centerObject(region, zoom=12)  # Centre our map on the region of interest
Map


Map(center=[-6.815979856908561, -59.11885924478193], controls=(WidgetControl(options=['position', 'transparent…

This time, we will pick two different scenes according to our dates.

In [73]:
# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
landsat8_collection = landsat8_collection.filterBounds(region)

# Get least cloudy image within first date bounds
image_1 = landsat8_collection \
            .filterDate(date_1_start, date_1_end) \
            .filter(ee.Filter.contains('.geo', region)) \
            .sort('CLOUD_COVER').first().clip(region)
date_1 = image_1.get('DATE_ACQUIRED').getInfo()

# Get least cloudy image within first date bounds
image_2 = landsat8_collection \
            .filterDate(date_2_start, date_2_end) \
            .filter(ee.Filter.contains('.geo', region)) \
            .sort('CLOUD_COVER').first().clip(region)
date_2 = image_2.get('DATE_ACQUIRED').getInfo()

# Print out date, for reference
print(f'Selected images on dates {date_1} and {date_2}')
# print(image_1)
# print(image_2)


Selected images on dates 2022-06-19 and 2023-06-22


In [74]:
Map = geemap.Map()  # Create empty map

max_reflectance = 0.25 # Set the upper limit of reflectance to visualise.
                       # Play with this value (between 0-1) to see what it
                       # does. It will need to be higher for snowy/icy
                       # scenes

visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image_1, visParams, date_1)
Map.addLayer(image_2, visParams, date_2)

Map.centerObject(region, zoom=12)
Map

Map(center=[-6.815979856908561, -59.11885924478193], controls=(WidgetControl(options=['position', 'transparent…

In [75]:
# type_of_ndi = 'NBR'  # type of NDI calculated, for filename purposes
# r_high = 'B5'  # relevant band name for r_high
# r_low = 'B6'   # relevant band name for r_low

type_of_ndi = 'NDVI'  # type of NDI calculated, for filename purposes
r_high = 'B5'  # relevant band name for r_high
r_low = 'B4'   # relevant band name for r_low

ndi_1 = image_1.normalizedDifference([r_high, r_low]).rename(type_of_ndi)
ndi_2 = image_2.normalizedDifference([r_high, r_low]).rename(type_of_ndi)

ndi_diff = ndi_2.subtract(ndi_1).rename(f'{type_of_ndi}_diff')


In [76]:
Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.15
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image_1, visParams, date_1)
Map.addLayer(image_2, visParams, date_2)

# Display NDI
visParams = {'bands': [type_of_ndi], 'min': -1, 'max': 1, 'palette': ['brown', 'white', 'green']}
Map.addLayer(ndi_1, visParams, f'{type_of_ndi} {date_1}')
Map.addLayer(ndi_2, visParams, f'{type_of_ndi} {date_2}')

visParams = {'bands': [f'{type_of_ndi}_diff'], 'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(ndi_diff, visParams, f'Change in {type_of_ndi}, {date_1} {date_2}')

Map.centerObject(region, zoom=12)
Map


Map(center=[-6.815979856908561, -59.11885924478193], controls=(WidgetControl(options=['position', 'transparent…

In [77]:
# Sample 10,000 pixels within the NDI image
sample_pixels = ndi_diff.sample(region, numPixels=10000)

# Set labels for the graph
labels = {
    "title": 'Distribution of ΔNDI values within image',
    "xlabel": f'Δ{type_of_ndi} values',
    "ylabel": 'Pixel count',
}

# Construct the histogram
chart.feature_histogram(sample_pixels, f'{type_of_ndi}_diff', **labels)


In [83]:
# manually set a threshold
threshold = -0.05

# threshold the image to where greater than threshold.
# note that you could change from greater than to less than by
# changing 'gt' to 'lt'.
ndi_threshold = ndi_diff.lt(threshold)

# 'Mask' the data, showing only regions beyond the threshold.
ndi_threshold = ndi_threshold.updateMask(ndi_threshold.neq(0))


In [84]:
Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.2
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image_1, visParams, date_1)
Map.addLayer(image_2, visParams, date_2)

# Display NDI
visParams = {'bands': [f'{type_of_ndi}_diff'], 'palette': ['white', 'red'], 'opacity': 0.3}
Map.addLayer(ndi_threshold, visParams, type_of_ndi)

Map.centerObject(region, zoom=12)
Map

Map(center=[-6.815979856908561, -59.11885924478193], controls=(WidgetControl(options=['position', 'transparent…

In [94]:
# Get total area of initial search region
aoi_area_km2 = region.area(maxError=1).getInfo() / 1e6

# Get total thresholded
thresh_area_km = ndi_threshold.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = ndi_threshold.geometry(),
    scale = 30,
    maxPixels = 1e9
  ).getInfo()['NDVI_diff'] * 30 * 30 / 1e6

# Print the results
print(f'Total scene area: {aoi_area_km2:.2f} km2')
print(f'Total classified area: {thresh_area_km:.2f} km2')


Total scene area: 397.80 km2
Total classified area: 49.02 km2


# Export

In [96]:
# You can edit this variables
folder = 'scires_project_2C'


In [ ]:
# Construct the filename automatically
date_1_string = image_1.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_1_string + '_image'

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image_1.select(['B4', 'B3', 'B2', 'B5', 'B6']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


In [ ]:
# Construct the filename automatically
date_2_string = image_2.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_2_string + '_image'

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image_2.select(['B4', 'B3', 'B2', 'B5', 'B6']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_1_string + '_' + type_of_ndi

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi_1,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_2_string + '_' + type_of_ndi

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi_2,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_1_string + '_' + date_2_string + '_' + type_of_ndi + '_diff'

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi_diff,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')

In [ ]:
# Construct the filename automatically
filename = location_name + '_' + date_1_string + '_' + date_2_string + '_' + type_of_ndi + '_diff_thresh_' + str(threshold)

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi_threshold,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')